In [1]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
import requests
import os

## ***Build Weather Tools using realtime api***

In [2]:
load_dotenv()
weather_api = os.getenv("WEATHER_API")
groq_api_key = os.getenv("GROQ_API_KEY")

In [3]:
llm = ChatGroq(model_name = "meta-llama/llama-4-scout-17b-16e-instruct", groq_api_key=groq_api_key)

## *Define the tools*

In [4]:
@tool
def get_weather_data(city: str) -> str:
    """
    This function fetches the current weather data for a given city
    """
    url = f"http://api.weatherstack.com/current?access_key={weather_api}&query={city}"
    response = requests.get(url)
    return response.json()

In [5]:
get_weather_data.invoke(
    {
        "city": "Dhaka"
    }
)

{'request': {'type': 'City',
  'query': 'Dhaka, Bangladesh',
  'language': 'en',
  'unit': 'm'},
 'location': {'name': 'Dhaka',
  'country': 'Bangladesh',
  'region': '',
  'lat': '23.723',
  'lon': '90.409',
  'timezone_id': 'Asia/Dhaka',
  'localtime': '2025-05-05 10:55',
  'localtime_epoch': 1746442500,
  'utc_offset': '6.0'},
 'current': {'observation_time': '04:55 AM',
  'temperature': 33,
  'weather_code': 113,
  'weather_icons': ['https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0001_sunny.png'],
  'weather_descriptions': ['Sunny'],
  'astro': {'sunrise': '05:22 AM',
   'sunset': '06:28 PM',
   'moonrise': '12:21 PM',
   'moonset': '01:03 AM',
   'moon_phase': 'Waxing Gibbous',
   'moon_illumination': 54},
  'air_quality': {'co': '551.3',
   'no2': '51.06',
   'o3': '38',
   'so2': '29.785',
   'pm2_5': '67.895',
   'pm10': '76.22',
   'us-epa-index': '4',
   'gb-defra-index': '4'},
  'wind_speed': 7,
  'wind_degree': 135,
  'wind_dir': 'SE',
  'pressure': 100

In [6]:
search_tool = TavilySearchResults()

## Now time to define the Agents

In [7]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [8]:
prompt = hub.pull("hwchase17/react")
print(prompt.template)

c:\Users\Lenovo\anaconda3\envs\torchGPU\Lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [9]:
agent = create_react_agent(
    llm=llm,
    tools=[search_tool, get_weather_data],
    prompt=prompt
)

In [10]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool, get_weather_data],
    verbose=True
)

## Now time to Test my simple Agent

In [11]:
response = agent_executor.invoke(
    {
        "input": "Find the capital of Bangladesh, then find the its current weather condition today."
    }
)
print(response)



> Entering new AgentExecutor chain...


c:\Users\Lenovo\anaconda3\envs\torchGPU\Lib\site-packages\pydantic\main.py:1114: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
c:\Users\Lenovo\anaconda3\envs\torchGPU\Lib\site-packages\pydantic\main.py:1114: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
c:\Users\Lenovo\anaconda3\envs\torchGPU\Lib\site-packages\pydantic\main.py:1114: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in

To find the capital of Bangladesh and then its current weather condition, I should first determine the capital of Bangladesh.

Action: tavily_search_results_json
Action Input: "capital of Bangladesh"[{'url': 'http://www.discoverybangladesh.com/capital.html', 'content': "Dhaka - The Capital of Bangladesh\n\nDhaka is the capital and largest city of Bangladesh. With its colorful history and rich cultural traditions, Dhaka is known the world over as the city of mosques and muslin. Its fame attracted travelers from far and near throughout the ages. Today it has grown into a mega city of about 8.5 million people, with an area of about 1353 sq. km. becoming the hub of the nation's industrial, commercial, cultural, educational and political activities. [...] rewarding experience. [...] Ph: 880-2-8914416."}, {'url': 'https://en.wikipedia.org/wiki/Dhaka', 'content': "As the capital of the People's Republic of Bangladesh, Dhaka is home to numerous state and diplomatic institutions. The Bangabhaba

c:\Users\Lenovo\anaconda3\envs\torchGPU\Lib\site-packages\pydantic\main.py:1114: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
c:\Users\Lenovo\anaconda3\envs\torchGPU\Lib\site-packages\pydantic\main.py:1114: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
c:\Users\Lenovo\anaconda3\envs\torchGPU\Lib\site-packages\pydantic\main.py:1114: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in

Thought: The search results confirm that the capital of Bangladesh is Dhaka. Now, I need to find the current weather condition in Dhaka.

Action: get_weather_data
Action Input: "Dhaka"{'request': {'type': 'City', 'query': 'Dhaka, Bangladesh', 'language': 'en', 'unit': 'm'}, 'location': {'name': 'Dhaka', 'country': 'Bangladesh', 'region': '', 'lat': '23.723', 'lon': '90.409', 'timezone_id': 'Asia/Dhaka', 'localtime': '2025-05-05 10:55', 'localtime_epoch': 1746442500, 'utc_offset': '6.0'}, 'current': {'observation_time': '04:55 AM', 'temperature': 33, 'weather_code': 113, 'weather_icons': ['https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0001_sunny.png'], 'weather_descriptions': ['Sunny'], 'astro': {'sunrise': '05:22 AM', 'sunset': '06:28 PM', 'moonrise': '12:21 PM', 'moonset': '01:03 AM', 'moon_phase': 'Waxing Gibbous', 'moon_illumination': 54}, 'air_quality': {'co': '551.3', 'no2': '51.06', 'o3': '38', 'so2': '29.785', 'pm2_5': '67.895', 'pm10': '76.22', 'us-epa-ind

c:\Users\Lenovo\anaconda3\envs\torchGPU\Lib\site-packages\pydantic\main.py:1114: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
c:\Users\Lenovo\anaconda3\envs\torchGPU\Lib\site-packages\pydantic\main.py:1114: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
c:\Users\Lenovo\anaconda3\envs\torchGPU\Lib\site-packages\pydantic\main.py:1114: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in

In [12]:
response['output']

'The capital of Bangladesh is Dhaka, and its current weather condition today is sunny with a temperature of 33°C.'